In [3]:
# --- Data Loading ---
import gzip
import pickle
import numpy as np

def load_data():
    f = gzip.open('/content/mnist.pkl.gz', 'rb')  # adjust path if necessary
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return training_data, validation_data, test_data

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return training_data, validation_data, test_data


In [12]:
import random
import numpy as np

# --- Cost Function ---
class CrossEntropyCost:
    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a) - (1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        return (a - y)

class LogLikelihoodCost:
    @staticmethod
    def fn(a, y):
        return -np.sum(np.nan_to_num(y * np.log(a)))

    @staticmethod
    def delta(z, a, y):
        return a - y  # same as cross-entropy, with softmax



# --- Network Class ---
class Network:
    def __init__(self, sizes, cost=CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost = cost

    def large_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def default_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x) / np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
      for i in range(len(self.biases) - 1):
          a = sigmoid(np.dot(self.weights[i], a) + self.biases[i])
      z = np.dot(self.weights[-1], a) + self.biases[-1]
      a = softmax(z)  # softmax only at the output layer
      return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda=0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):

        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)

        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, n)

            print(f"Epoch {j} training complete")

            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print(f"Cost on training data: {cost}")

            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print(f"Accuracy on training data: {accuracy} / {n}")

            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print(f"Cost on evaluation data: {cost}")

            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print(f"Accuracy on evaluation data: {accuracy} / {n_data}")

        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [(1 - eta * (lmbda / n)) * w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        # Initialize gradients
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # Forward pass
        activation = x
        activations = [x]
        zs = []

        for i in range(len(self.biases) - 1):
            z = np.dot(self.weights[i], activation) + self.biases[i]
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        # Output layer (softmax)
        z = np.dot(self.weights[-1], activation) + self.biases[-1]
        zs.append(z)
        activation = softmax(z)
        activations.append(activation)

        # Backward pass
        delta = self.cost.delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].T)

        return (nabla_b, nabla_w)



    def accuracy(self, data, convert=False):
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y) for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y) / len(data)
        cost += 0.5 * (lmbda / len(data)) * sum(np.linalg.norm(w)**2 for w in self.weights)
        return cost


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

def softmax(z):
    exp_z = np.exp(z - np.max(z))  # subtract max for numerical stability
    return exp_z / np.sum(exp_z, axis=0)



In [13]:
training_data, validation_data, test_data = load_data_wrapper()

net = Network([784, 30, 10], cost=LogLikelihoodCost)
net.large_weight_initializer()

net.SGD(training_data, 30, 10, 0.5,
        evaluation_data=test_data,
        monitor_evaluation_accuracy=True)


Epoch 0 training complete
Accuracy on evaluation data: 9037 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 9212 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9280 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9318 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9377 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9455 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9474 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9444 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9430 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9473 / 10000
Epoch 10 training complete
Accuracy on evaluation data: 9446 / 10000
Epoch 11 training complete
Accuracy on evaluation data: 9470 / 10000
Epoch 12 training complete
Accuracy on evaluation data: 9512 / 10000
Epoch 13 training complete
Accuracy on evaluation data: 9458 / 10000
Epoch 14 training complete
Accuracy on evalu

([],
 [9037,
  9212,
  9280,
  9318,
  9377,
  9455,
  9474,
  9444,
  9430,
  9473,
  9446,
  9470,
  9512,
  9458,
  9499,
  9507,
  9470,
  9525,
  9532,
  9525,
  9504,
  9493,
  9528,
  9553,
  9496,
  9517,
  9483,
  9500,
  9527,
  9515],
 [],
 [])